<center>

# <u>**Arbitage Betting**</u>

</center>

Arbitrage betting is a strategy where a bettor exploits differences in odds among various bookmakers to guarantee a profit. By placing bets on all possible outcomes of an event, the bettor ensures a positive return regardless of the actual outcome. This involves capitalizing on pricing discrepancies between bookmakers. It's a quick and precise approach, aiming for risk-free gains.

### Description:

This arbitage alorithm considers 2 betting games:

1. Over/Under (Totals) betting predicts if a game's final score will exceed (Over) or fall below (Under) a set score by the bookmaker, involving forecasting the total points or goals in a game
2. Head-to-Head (H2H) betting, individuals place bets on the prospective victory of either the home or away team, with the provision to bet on a draw if applicable.

The final result is 2 pandas dataframes projecting all of the arbitage possibilities in a ranked order of with the higher Return On Investment (ROI).

### Data: 


The data collected are live data taken from the [ODDS API](https://the-odds-api.com/) and we collect only the 8 upcoming/live games.


## ***To Run the algorithm:***
1. Firstly, you need to get your own free [ODDS API key](https://the-odds-api.com/#get-access).
2. Once you get your API-KEY, you should place it in the .env file. 
3. Secondly, press the **Run ALL** option from the top tool bar top instantiate and run all cells in the notebook.
4. Then press [here](#run-section) to go to the final cell to look at the results. 
5. If you want to re-run the algorithm just run the last cell.

Note: Remember there are limited API requests per month.

In [60]:
# Install necessary packages first
%pip install pandas
%pip install dotenv
%pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [62 lines of output]
        error: subprocess-exited-with-error
      
        Ã— python setup.py egg_info did not run successfully.
        â”‚ exit code: 1
        â•°â”€> [16 lines of output]
            Traceback (most recent call last):
              File "<string>", line 2, in <module>
              File "<pip-setuptools-caller>", line 14, in <module>
              File "C:\Users\stama\AppData\Local\Temp\pip-wheel-c0ofk5pb\distribute_523335ea520a4fa5960e93e47124a24d\setuptools\__init__.py", line 2, in <module>
                from setuptools.extension import Extension, Library
              File "C:\Users\stama\AppData\Local\Temp\pip-wheel-c0ofk5pb\distribute_523335ea520a4fa5960e93e47124a24d\setuptools\extension.py", line 5, in <module>
                from setuptools.dist import _get_unpatched
              File "C:\Users\stama\AppData\Local\Temp\pip-wheel-c0of

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [61]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta
import pytz

load_dotenv()

True

In [62]:
def get_api_data_request():

    # Get a free API key at https://api.the-odds-api.com/
    API_KEY = os.getenv('API-KEY')

    SPORT = 'upcoming' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports

    REGIONS = 'uk' # uk | us | eu | au. Multiple can be specified if comma delimited

    MARKETS =  'totals,h2h' # h2h | spreads | totals. Multiple can be specified if comma delimited

    ODDS_FORMAT = 'decimal' # decimal | american

    DATE_FORMAT = 'iso' # iso | unix


    # Set the time zone to UK
    uk_timezone = pytz.timezone('Europe/London')

    uk_current_time = datetime.now(uk_timezone).strftime('%Y-%m-%dT%H:%M:%SZ')

    COMMENCE_TIME_FROM = uk_current_time



    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
    #
    # Now get a list of live & upcoming games for the sport you want, along with odds for different bookmakers
    # This will deduct from the usage quota
    # The usage quota cost = [number of markets specified] x [number of regions specified]
    # For examples of usage quota costs, see https://the-odds-api.com/liveapi/guides/v4/#usage-quota-costs
    #
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

    odds_response = requests.get(
        f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
        params={
            'api_key': API_KEY,
            'regions': REGIONS,
            'markets': MARKETS,
            'oddsFormat': ODDS_FORMAT,
            'dateFormat': DATE_FORMAT,
            'commenceTimeFrom': COMMENCE_TIME_FROM,
        }
    )

    if odds_response.status_code != 200:
        print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')
        return odds_response.status_code
    else:
        odds_json = odds_response.json()
        print("API request Successfull!")
        print("Data Received!")
        print('Number of events (matches):', len(odds_json))

        odds_df = pd.DataFrame(odds_json)

        # Check the usage quota
        print('Remaining requests', odds_response.headers['x-requests-remaining'])
        print('Used requests', odds_response.headers['x-requests-used'])
        return odds_df

### Data manipulation:

The data_manipulation function is responsible for:
- Creating a dataframe where each row represents a bookmaker for each match.
- Distinguishing which events have an applicable h2h (with or without Draw) and totals betting.
- And performs adds null values to not applicable bettings.

In [63]:
def data_manipulation(odds_df):
    print("Data Cleaning in process...")
    odds_list = []

    uk_timezone = pytz.timezone('Europe/London')

    uk_current_time = datetime.now(uk_timezone).strftime('%Y-%m-%dT%H:%M:%SZ')

    for i, game in odds_df.iterrows():
        for bookmaker in game['bookmakers']:
        
            book = {}
            book['id'] = game['id']
            book['sport'] = game['sport_title']
            book['home_team'] = game['home_team']
            book['away_team'] = game['away_team']
            book['commence_time'] = game['commence_time']
            book['bookmaker'] = bookmaker['title']

                
            time_diff = datetime.strptime(uk_current_time, '%Y-%m-%dT%H:%M:%SZ')-datetime.strptime(bookmaker['last_update'], '%Y-%m-%dT%H:%M:%SZ')
            # Extract components
            hours, remainder = divmod(time_diff.seconds, 3600)
            minutes, seconds = divmod(remainder, 60)

            # Format the result with letters
            result_str = f"{hours}H {minutes}M {seconds}S"
            book['last-update-ago'] = result_str
        
            # If totals & h2h odds are applicable get them
            if len(bookmaker['markets']) == 2 and bookmaker['markets'][1]['key'] == 'totals':
                book['h2h_home_odds'] = bookmaker['markets'][0]['outcomes'][0]['price']
                book['h2h_away_odds'] = bookmaker['markets'][0]['outcomes'][1]['price']
                if len(bookmaker['markets'][0]['outcomes']) == 3:
                    book['h2h_draw_odds'] = bookmaker['markets'][0]['outcomes'][2]['price']
                else:
                    book['h2h_draw_odds'] = None

                book['total_odds_over'] = bookmaker['markets'][1]['outcomes'][0]['price']  
                book['total_point_over'] = bookmaker['markets'][1]['outcomes'][0]['point']
                book['total_odds_under'] = bookmaker['markets'][1]['outcomes'][1]['price']
                book['total_point_under'] = bookmaker['markets'][1]['outcomes'][1]['point']
            else:
                if bookmaker['markets'][0]['key'] == 'h2h':
                    # Get h2h
                    book['h2h_home_odds'] = bookmaker['markets'][0]['outcomes'][0]['price']
                    book['h2h_away_odds']= bookmaker['markets'][0]['outcomes'][1]['price']

                    # Cases were Draw is not applicable like (e.g. tennis)
                    if len(bookmaker['markets'][0]['outcomes'])==3:
                        book['h2h_draw_odds'] = bookmaker['markets'][0]['outcomes'][2]['price']
                    else:
                        book['h2h_draw_odds'] = None
                        
                    # Initialise totals as None
                    book['total_odds_over'] = None
                    book['total_point_over'] = None
                    book['total_odds_under'] = None
                    book['total_point_under'] = None
                elif bookmaker['markets'][0]['key'] == 'totals':
                    book['total_odds_over'] = bookmaker['markets'][0]['outcomes'][0]['price']
                    book['total_point_over'] = bookmaker['markets'][0]['outcomes'][0]['point']
                    book['total_odds_under'] = bookmaker['markets'][0]['outcomes'][1]['price']
                    book['total_point_under'] = bookmaker['markets'][0]['outcomes'][1]['point']

                    book['h2h_home_odds'] = None
                    book['h2h_away_odds'] = None
                    book['h2h_draw_odds'] = None

            print(book)
            odds_list.append(book)
    print("Data Cleaning Successfull.")
    print("There ", len(odds_list)," different odds.")
    return pd.DataFrame(odds_list)



### Performing Arbitage:

This section performs calculations in estimating arbitage odds:

1. [Firstly](#Arbitage-Totals-(Under/Over)) we calculate the arbitage odds of the Totals (Over/Under) bettings.
2. [Secondly](#Arbitage-h2h) we calculate the arbitage odds of the H2H bettings.

End result are 2 ranked (based on highest ROI) dataframes with all the arbitage possibilities for both totals and h2h. If there are not arbitage possibilities you will be notified with a message.

### Arbitage Totals (Under/Over):

In [64]:
# Calculate arbitage
def calculate_arbitage_totals(over, under, bet=100):

    # The total stake probability
    total_stake = 1/over['total_odds_over'] + 1/under['total_odds_under']

    # Calculate the amount of bet needed for each outcome
    over_bet = ((1/over['total_odds_over'])/total_stake) * bet
    under_bet = ((1/under['total_odds_under'])/total_stake) * bet

    over['bet_amount'] = round(over_bet, 2)
    under['bet_amount'] = round(under_bet, 2)

    # Remove Unused columns
    over = over.drop('total_odds_under')
    under = under.drop('total_odds_over')
    
    # Calculate the arbitage profit
    profit = over_bet*over['total_odds_over']
    
    # Calculate the Revenue Over Income (ROI)
    roi_perc = ((profit-bet)*100)/bet

    return { "match_id": over['id'], 'commence_time':over['commence_time'], 'sport' : over['sport'],
             'home_team': over['home_team'], 'away_team':over['away_team'],
             'over_bookmaker':over['bookmaker'], 'odds_over': over['total_odds_over'], 'over_bet': over['bet_amount'], 
             'under_bookmaker':under['bookmaker'], 'odds_under': under['total_odds_under'], 'under_bet': under['bet_amount'],
             'over_under_point': over['total_point_over'],'initial_bet' : bet,"revenue": profit, "roi%": round(roi_perc,2)}



    


def get_arbitage_possibilities_totals(df, points, initial_bet):

    # Check for every Over/Under possibilities for totals arbitage and 
    # Find bookmakers that use the same betting points
    totals_arbitage_list = []
    for point in points:

        # Get group of bookmakers that used the same betting points
        grouped = df[(df['total_point_over'] == point) & (df['total_point_under'] == point)]

        for i, row_1 in grouped.iterrows():
            # Allow only combinations that are equal to the current point
            if row_1['total_point_over'] != point:
                continue

            for j, row_2 in grouped.iterrows():
                # Allow only combinations that are equal to the current point
                if i>=j or row_2['total_point_over'] != point:
                    continue

                # Check for arbitage possibility between home team (Over) and away team (Under)
                sum_prob = 1/row_1['total_odds_over'] + 1/row_2['total_odds_under']
                if sum_prob<1:
                    totals_arbitage_list.append(calculate_arbitage_totals(row_1, row_2, initial_bet))
                    
                # Check for arbitage possibility between home team (Under) and away team (Over)
                sum_prob = 1/row_1['total_odds_under'] + 1/row_2['total_odds_over']
                if sum_prob<1:
                    totals_arbitage_list.append(calculate_arbitage_totals(row_2, row_1, initial_bet))
                
    
    return pd.DataFrame(totals_arbitage_list)

### Arbitage h2h 

- The possible outcomes are Home team Win, Away tema win or Draw

In [65]:
# Calculate the h2h probabiltity of eaither home and away team to win
def calculate_arbitage_h2h(home, away, bet=100):
    # The total stake probability
    total_stake = 1/home['h2h_home_odds'] + 1/away['h2h_away_odds']

    # Calculate the amount of bet needed for each outcome
    away_bet = ((1/away['h2h_away_odds'])/total_stake) * bet
    home_bet = ((1/home['h2h_home_odds'])/total_stake) * bet

    home['bet_amount'] = round(home_bet, 2)
    away['bet_amount'] = round(away_bet, 2)

    # Remove Unused columns
    home = home.drop(['h2h_away_odds'])
    away = away.drop(['h2h_home_odds'])
   
    # Calculate the arbitage profit
    profit = home_bet*home['h2h_home_odds']
    
    # Calculate the Revenue Over Income (ROI)
    roi_perc = ((profit-bet)*100)/bet

    
    return {"match_id": home['id'], 'commence_time':home['commence_time'], 'sport' : home['sport'],
             'home_team':home['home_team'], 'away_team': away['away_team'],
            'home_bookmaker':home['bookmaker'], 'home_odds': home['h2h_home_odds'], 
            'home_bet': home['bet_amount'], 'home_odds_last_update':home['last-update-ago'],
            'away_bookmaker':away['bookmaker'], 'away_odds': away['h2h_away_odds'], 
            'away_bet': away['bet_amount'], 'away_odds_last_update':away['last-update-ago'],
            'draw_bookmaker':None, 'draw_odds': None, 'draw_bet': None,
            'initial_bet' : bet,"revenue": profit,"roi%": round(roi_perc,2)}



# Calculate arbitage 3 Outcome h2h arbitage for home/away team to Win or Draw
def calculate_arbitage_h2h_with_draw(home, away, draw, bet=100):
    # The total stake probability
    total_stake = 1/home['h2h_home_odds'] + 1/away['h2h_away_odds'] + 1/draw['h2h_draw_odds']

    # Calculate the amount of bet needed for each outcome
    home_bet = ((1/home['h2h_home_odds'])/total_stake) * bet
    away_bet = ((1/away['h2h_away_odds'])/total_stake) * bet
    draw_bet = ((1/draw['h2h_draw_odds'])/total_stake) * bet

    home['bet_amount'] = round(home_bet, 2)
    away['bet_amount'] = round(away_bet, 2)
    draw['bet_amount'] = round(draw_bet, 2)

    # Remove Unused columns
    home = home.drop(['h2h_away_odds', 'h2h_draw_odds'])
    away = away.drop(['h2h_home_odds', 'h2h_draw_odds'])
    draw = draw.drop(['h2h_home_odds', 'h2h_away_odds'])
    
    # Calculate the arbitage profit
    profit = home_bet*home['h2h_home_odds']    
    
    # Calculate the Revenue Over Income (ROI)
    roi_perc = ((profit-bet)*100)/bet
    
    return {"match_id": home['id'], 'commence_time':home['commence_time'], 'sport' : home['sport'],
            'home_team':home['home_team'], 'away_team': away['away_team'],
            'home_bookmaker':home['bookmaker'], 'home_odds': home['h2h_home_odds'], 
            'home_bet': home['bet_amount'], 'home_odds_last_update':home['last-update-ago'],
            'away_bookmaker':away['bookmaker'], 'away_odds': away['h2h_away_odds'], 
            'away_bet': away['bet_amount'], 'away_odds_last_update':away['last-update-ago'],
            'draw_bookmaker':draw['bookmaker'], 'draw_odds': draw['h2h_draw_odds'], 
            'draw_bet': draw['bet_amount'], 'draw_odds_last_update':draw['last-update-ago'],
            'initial_bet' : bet, "revenue": profit,"roi%": round(roi_perc,2)}

#def refactored_get_arbitage_possibilities_h2h(df, draw_outcome, initial_bet):


def get_arbitage_possibilities_h2h(df, draw_outcome, initial_bet):
    h2h_arbitage_list = []
    for index_1, row_1 in df.iterrows():
        for index_2, row_2 in df.iterrows():

            # Do not allow repeated bookmakers
            if index_1 >= index_2:
                continue
            else:
                # Check for 3 Outcome arbitage
                if draw_outcome==True:
                    for index_3, row_3 in df.iterrows():
                        # Do not allow repeated bookmakers
                        if index_1>=index_3 or index_2>=index_3:
                            continue
                        else:
                            # Get all possible combinations from the 3 unique bookmakers and check for arbitage
                            # row_1(home), row_2(away), row_3(draw)                          
                            sum_prob = 1/row_1['h2h_home_odds'] + 1/row_2['h2h_away_odds']+ 1/row_3['h2h_draw_odds']
                            if sum_prob<1:
                                h2h_arbitage_list.append(calculate_arbitage_h2h_with_draw(row_1, row_2, row_3, initial_bet))

                             # row_2(home), row_1(away), row_3(draw)   
                            sum_prob = 1/row_1['h2h_away_odds'] + 1/row_2['h2h_home_odds'] + 1/row_3['h2h_draw_odds']
                            if sum_prob<1:
                                h2h_arbitage_list.append(calculate_arbitage_h2h_with_draw(row_2, row_1, row_3, initial_bet))
                            
                            # row_3(home), row_2(away), row_1(draw)   
                            sum_prob = 1/row_1['h2h_draw_odds'] + 1/row_2['h2h_away_odds'] + 1/row_3['h2h_home_odds']
                            if sum_prob<1:
                                h2h_arbitage_list.append(calculate_arbitage_h2h_with_draw(row_3, row_2, row_1, initial_bet))

                            # row_3(home), row_1(away), row_2(draw)   
                            sum_prob = 1/row_1['h2h_away_odds'] + 1/row_2['h2h_draw_odds'] + 1/row_3['h2h_home_odds']
                            if sum_prob<1:
                                h2h_arbitage_list.append(calculate_arbitage_h2h_with_draw(row_3, row_1, row_2, initial_bet))
                            
                            # row_1(home), row_3(away), row_2(draw)   
                            sum_prob = 1/row_1['h2h_home_odds'] + 1/row_2['h2h_draw_odds'] + 1/row_3['h2h_away_odds']
                            if sum_prob<1:
                                h2h_arbitage_list.append(calculate_arbitage_h2h_with_draw(row_1, row_3, row_2, initial_bet))
                                                       
                            # row_2(home), row_3(away), row_1(draw)   
                            sum_prob = 1/row_1['h2h_draw_odds'] + 1/row_2['h2h_home_odds'] + 1/row_3['h2h_away_odds']
                            if sum_prob<1:
                                h2h_arbitage_list.append(calculate_arbitage_h2h_with_draw(row_2, row_3, row_1, initial_bet))
                else:
                    # Get all 2 Outcome h2h arbitage possibilities

                    # row_1(home), row_2(away)
                    sum_prob = 1/row_1['h2h_home_odds'] + 1/row_2['h2h_away_odds']
                    if sum_prob<1:
                        h2h_arbitage_list.append(calculate_arbitage_h2h(row_1, row_2, initial_bet))
                    
                    # row_2(home), row_1(away)
                    sum_prob = 1/row_1['h2h_away_odds'] + 1/row_2['h2h_home_odds']
                    if sum_prob<1:
                        #print('Abritage Prob found: ', sum_prob)
                        h2h_arbitage_list.append(calculate_arbitage_h2h(row_2, row_1, initial_bet))
                        #h2h_arbitage_list.append(calculate_arbitage(row_2, row_1))


    return pd.DataFrame(h2h_arbitage_list)

### Get Arbitage for each of the upcoming games:

In [66]:
# Get Arbitage for each bookmaker for each match

# Get a list of all bookmakers and their odds for each match
def arbitage(odds_df, initial_bet):
    print("Arbitage in process, might take a few moments....")
    # Group each odd to the match ID
    matches = list(odds_df.groupby('id'))
    totals_df = pd.DataFrame()
    h2h_df = pd.DataFrame()

    for match in matches:
        points = []
        match_id = match[0]
        match_df = match[1]
        # Totals Arbitage
        # Number of unique total betting points in the data
        set_points = list(set(match_df['total_point_over']))
        for point in set_points:
            if point not in points and pd.notna(point):
                points.append(point)
        # Concatinate all arbitage possibilities for each match
        totals_df = pd.concat([totals_df, get_arbitage_possibilities_totals(match_df, points, initial_bet)], ignore_index=True)

        # H2H Arbitage
        # Check if a draw outcome is suitable for h2h betting
        draw_outcome = False
        if not match_df['h2h_draw_odds'].isnull().values.all():
            draw_outcome = True
        # Concatinate all arbitage possibilities for each match
        h2h_df = pd.concat([h2h_df, get_arbitage_possibilities_h2h(match_df, draw_outcome, initial_bet)], ignore_index=True)

    # Remove duplicated rows and rank dataframes based on ROI
    if totals_df.empty==False:
        totals_df = totals_df.drop_duplicates().sort_values(by='roi%', ascending=False).reset_index().drop('index', axis=1, inplace=False)
    if h2h_df.empty==False:
        h2h_df = h2h_df.drop_duplicates().sort_values(by='roi%', ascending=False).reset_index().drop('index', axis=1, inplace=False)
    return totals_df, h2h_df
       

Below the High Level function that connects all the arbitage processes together along with display of the results.

In [67]:
def run_arbitage(initial_bet=100, top_10=True):

    # Start the arbitage process
    try:
        odds_df = get_api_data_request()
        odds_df = data_manipulation(odds_df)
        totals_df, h2h_df = arbitage(odds_df, initial_bet)
    except Exception as e:
        print(f"An error occurred: {e}")
        return e

    # Display results
    print("=="*50)
    if totals_df.empty==True and h2h_df.empty==True:
        print("There are no arbitage possibilities for both Totals (Over/Under) & H2H Betting. \n Try again later!")
    elif totals_df.empty==True:
        print("There are no arbitage possibilities for Totals (Over/Under) Betting. \n Try again later!")
        print("Arbitage possibilities for H2H: ")
        if top_10:
            display(h2h_df.head(10))
        else:
            display(h2h_df)
        print("=="*50)
    elif h2h_df.empty==True:
        print("There are no arbitage possibilities for H2H Betting. \n Try again later!")
        
        print("Arbitage possibilities for Totals (Over/Under): ")
        if top_10:
            display(totals_df.head(10))
        else:
            display(totals_df)
        print("=="*50)
    else:
        
        print("Arbitage possibilities for Totals (Over/Under): ")
        if top_10:
            display(totals_df.head(10))
        else:
            display(totals_df)
        print("=="*50)
        
        print("Arbitage possibilities for H2H: ")
        if top_10:
            display(h2h_df.head(10))
        else:
            display(h2h_df)
        print("=="*50)
    return totals_df, h2h_df


<center>

## **<u><a id="run-section" style="text-decoration: none;">Run Section</a></u>**

</center>

**Function:** 

run_arbitage(initial_bet=100, top_10=True)

**Parameters:**
- initial_bet (int):
    - The initial bet parameter indicates how much the user wants to bet in total. The higher the bet the higher the return. 
    - Default value is 100 (of any currency).
- top_10 (boolean): 
    - The top 10 parameter indicates if you want to just show the top 10 results of each arbitage returned value.
    - Default value is True

**Return:**
The function returns 2 dataframes the totals_df and the h2h_df which are the arbitage possibilities of totals and h2h betting respectively.

**Example:**
  run_arbitage(100, True)



**Note:** You can use the empty cell below to perform your own analysis.

In [68]:
# Set the display option to show all rows
pd.set_option('display.max_rows', None)

# Run
total_df, h2h_df = run_arbitage(initial_bet=100, top_10=True)

API request Successfull!
Data Received!
Number of events (matches): 9
Remaining requests 408
Used requests 92
Data Cleaning in process...
{'id': '92e537cbb828a96c9bb3fede86cd02c1', 'sport': 'NCAAB', 'home_team': 'Mercer Bears', 'away_team': 'South Alabama Jaguars', 'commence_time': '2023-12-06T16:00:00Z', 'bookmaker': 'Virgin Bet', 'last-update-ago': '0H 0M 25S', 'h2h_home_odds': 1.7, 'h2h_away_odds': 2.15, 'h2h_draw_odds': None, 'total_odds_over': 1.85, 'total_point_over': 132.5, 'total_odds_under': 1.85, 'total_point_under': 132.5}
{'id': '92e537cbb828a96c9bb3fede86cd02c1', 'sport': 'NCAAB', 'home_team': 'Mercer Bears', 'away_team': 'South Alabama Jaguars', 'commence_time': '2023-12-06T16:00:00Z', 'bookmaker': 'LiveScore Bet', 'last-update-ago': '0H 0M 6S', 'h2h_home_odds': 1.57, 'h2h_away_odds': 2.38, 'h2h_draw_odds': None, 'total_odds_over': 1.91, 'total_point_over': 133.5, 'total_odds_under': 1.83, 'total_point_under': 133.5}
{'id': '92e537cbb828a96c9bb3fede86cd02c1', 'sport': 'NC

,match_id,commence_time,sport,home_team,away_team,home_bookmaker,home_odds,home_bet,home_odds_last_update,away_bookmaker,away_odds,away_bet,away_odds_last_update,draw_bookmaker,draw_odds,draw_bet,initial_bet,revenue,roi%
0,92e537cbb828a96c9bb3fede86cd02c1,2023-12-06T16:00:00Z,NCAAB,Mercer Bears,South Alabama Jaguars,Virgin Bet,1.70,41.16,0H 0M 25S,Casumo,2.43,58.84,0H 4M 50S,None,None,None,100,69.975787,-30.02
1,92e537cbb828a96c9bb3fede86cd02c1,2023-12-06T16:00:00Z,NCAAB,Mercer Bears,South Alabama Jaguars,Virgin Bet,1.70,41.16,0H 0M 25S,Grosvenor,2.43,58.84,0H 4M 27S,None,None,None,100,69.975787,-30.02
2,44183ff08b763185f5636f1dddbefe4c,2023-12-06T17:30:00Z,One Day Internationals,West Indies,England,Betfair,1.53,34.54,0H 0M 52S,Casumo,2.90,65.46,0H 0M 43S,None,None,None,100,52.841986,-47.16
3,44183ff08b763185f5636f1dddbefe4c,2023-12-06T17:30:00Z,One Day Internationals,West Indies,England,Betfair,1.53,34.54,0H 0M 52S,Unibet,2.90,65.46,0H 0M 24S,None,None,None,100,52.841986,-47.16
4,44183ff08b763185f5636f1dddbefe4c,2023-12-06T17:30:00Z,One Day Internationals,West Indies,England,Betfair,1.53,34.54,0H 0M 52S,Grosvenor,2.90,65.46,0H 0M 11S,None,None,None,100,52.841986,-47.16


In [69]:
# Use this area for further analysis or usage...